In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

gpu PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
memory growth: True


In [2]:
#https://www.tensorflow.org/tutorials/text/text_classification_rnn

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer


train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
submit_sample_data = pd.read_csv('sample_submission.csv')

### Little bit of analyse and preprocessing

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
keyword_nan = train_data["keyword"].isna().sum()
location_nan = train_data["location"].isna().sum()
print("Keyword NaN's: {} \nLocation NaN's: {}".format(keyword_nan, location_nan))

Keyword NaN's: 61 
Location NaN's: 2533


In [7]:
train_data['keyword'].fillna('not-defined', inplace=True)
train_data['location'].fillna('not-defined', inplace=True)

In [8]:
keyword_nan = train_data["keyword"].isna().sum()
location_nan = train_data["location"].isna().sum()
print("Keyword NaN's: {} \nLocation NaN's: {}".format(keyword_nan, location_nan))

Keyword NaN's: 0 
Location NaN's: 0


In [9]:
train_data.head()

,id,keyword,location,text,target
0,1,not-defined,not-defined,Our Deeds are the Reason of this #earthquake M...,1
1,4,not-defined,not-defined,Forest fire near La Ronge Sask. Canada,1
2,5,not-defined,not-defined,All residents asked to 'shelter in place' are ...,1
3,6,not-defined,not-defined,"13,000 people receive #wildfires evacuation or...",1
4,7,not-defined,not-defined,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
y_train = train_data["target"]
X_train = train_data.drop(columns=["target", "id"])

### TF text to vector

In [11]:
# res = text_word_tokenizer.texts_to_sequences(X_train)
# res = tf.keras.preprocessing.sequence.pad_sequences(res, padding='post')
# res